In [ ]:
# Import necessary libraries for Slack integration, data handling, and machine learning
import re
import time
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import cryptpandas as crp
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Slack Bot Configuration
SLACK_BOT_TOKEN = "BOT_TOKEN"  # Replace with your bot token
CHANNEL_NAME = "#the-challenge"  # Slack channel to monitor
AUTHORIZED_USER_ID = "ID"  # Authorized user to trigger the bot actions

# Initialize Slack WebClient
client = WebClient(token=SLACK_BOT_TOKEN)

# Function to retrieve the Slack channel ID from the channel name
def get_channel_id(channel_name):
    try:
        response = client.conversations_list()
        for channel in response["channels"]:
            if channel["name"] == channel_name.lstrip("#"):
                return channel["id"]
    except SlackApiError as e:
        print(f"Error fetching channels: {e.response['error']}")
    return None

# Extract file ID and password from Slack message
def extract_id_and_password(message):
    match = re.search(r"release_(\d+)\.crypt.*?passcode is '(.*?)'", message)
    if match:
        file_id = match.group(1)
        password = match.group(2)
        return file_id, password
    return None

# Monitor Slack channel for new dataset release messages
def monitor_channel(channel_id, latest_timestamp):
    datasets = []
    try:
        response = client.conversations_history(channel=channel_id, oldest=latest_timestamp, limit=100)
        for message in reversed(response["messages"]):
            if message.get("user") == AUTHORIZED_USER_ID and "Data has just been released" in message["text"]:
                file_id, password = extract_id_and_password(message["text"])
                if file_id and password:
                    datasets.append((message["ts"], file_id, password))
    except SlackApiError as e:
        print(f"Error reading channel history: {e.response['error']}")
    return datasets

# Load encrypted dataset using the provided password
def load_dataset(file_id, password):
    try:
        file_name = f"release_{file_id}.crypt"
        file_path = f"./{file_name}"
        print(f"Loading dataset: {file_name}")
        X = crp.read_encrypted(path=file_path, password=password)
        print(f"Dataset loaded successfully. Shape: {X.shape}")
        return X
    except Exception as e:
        print(f"Error loading dataset {file_name}: {e}")
        return None

# Create lagged features for time-series prediction
def create_lagged_features(series, lag=1):
    lagged_data = pd.concat([series.shift(i) for i in range(lag, 0, -1)], axis=1)
    lagged_data.columns = [f'lag_{i}' for i in range(lag, 0, -1)]
    return lagged_data

# Apply linear regression to predict and optimize portfolio
def compute_predictions(X):
    df = X.copy()
    lag = 5
    predictions = {}

    for col in df.columns:
        lagged_features = create_lagged_features(df[col], lag)
        lagged_features['target'] = df[col]
        lagged_features = lagged_features.dropna()
        
        X_train, X_test, y_train, y_test = train_test_split(
            lagged_features.drop('target', axis=1),
            lagged_features['target'],
            test_size=0.2,
            shuffle=False
        )
        
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        next_input = X_test.iloc[-1:].values
        next_prediction = model.predict(next_input)
        
        predictions[col] = next_prediction[0]

    def normalize_positions(pos_dict):
        pos = pd.Series(pos_dict).replace([np.inf, -np.inf], np.nan).fillna(0)
        abs_sum = pos.abs().sum()
        if abs_sum > 0:
            pos = pos / abs_sum
        pos = pos.clip(-0.1, 0.1)
        if pos.abs().sum() > 0:
            pos = pos / pos.abs().sum()
        return pos

    def get_submission_dict(pos_dict, team_name="Jean Trading 69", passcode="JeanForTheWin"):
        positions = normalize_positions(pos_dict)
        return {**positions.to_dict(), "team_name": team_name, "passcode": passcode}

    return get_submission_dict(predictions)

# Automate Google Form submission
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium_stealth import stealth
from fake_useragent import UserAgent

LOGIN_URL = "https://accounts.google.com/signin"
GOOGLE_FORM_URL = "https://docs.google.com/forms/d/e/1FAIpQLSeUYMkI5ce18RL2aF5C8I7mPxF7haH23VEVz7PQrvz0Do0NrQ/viewform"

options = webdriver.ChromeOptions()
ua = UserAgent()
options.add_argument(f'user-agent={ua.random}')
driver = webdriver.Chrome(options=options)
stealth(driver, languages=["en-US", "en"], vendor="Google Inc.", platform="Win32", webgl_vendor="Intel Inc.", renderer="Intel Iris OpenGL Engine", fix_hairline=True)

# Fill and submit Google Form with predictions
def submit_to_google_form(submission_dict):
    driver.get(GOOGLE_FORM_URL)
    textarea = driver.find_element(By.XPATH, "/html/body/div/div[2]/form/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div[1]/div[2]/textarea")
    textarea.clear()
    textarea.send_keys(str(submission_dict).replace("'", '"'))
    print("Form filled successfully. Please review and submit manually.")

# Main workflow to coordinate Slack monitoring, prediction, and form submission
def main():
    print("Initializing monitoring...")
    channel_id = get_channel_id(CHANNEL_NAME)
    latest_timestamp = "0"
    processed_datasets = set()

    while True:
        datasets = monitor_channel(channel_id, latest_timestamp)
        for ts, file_id, password in datasets:
            if file_id not in processed_datasets:
                X = load_dataset(file_id, password)
                if X is not None:
                    submission = compute_predictions(X)
                    submit_to_google_form(submission)
                processed_datasets.add(file_id)
                latest_timestamp = ts
        time.sleep(10)

main()
